In [25]:
import pandas as pd
import requests
import psycopg2
from datetime import datetime


In [27]:
url = "https://arbeitnow.com/api/job-board-api"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

data = response.json()


In [28]:
df = pd.DataFrame(data['data'])

In [29]:
df["tags"] = df["tags"].apply(lambda x: ", ".join(x))

In [30]:
df["job_types"] = df["job_types"].apply(lambda x: ", ".join(x))

In [31]:
df["created_at"] = df["created_at"].apply(datetime.fromtimestamp)

In [32]:
df

,slug,company_name,title,description,remote,url,tags,job_types,location,created_at
0,account-manager-berlin-9697,W&W Human Resources - Ivonne Wresch -,Account Manager m/w/d,<p>Möchtest Du als Account Manager m/w/d für e...,False,https://www.arbeitnow.com/jobs/companies/ww-hu...,Customer Service,,Berlin,2025-02-16 20:49:06
1,telefonischer-kundenberater-call-center-agent-...,W&W Human Resources - Ivonne Wresch -,Telefonischer Kundenberater / Call Center Agen...,<p>Willst Du für ein Vergleichsportal als Kund...,False,https://www.arbeitnow.com/jobs/companies/ww-hu...,Customer Service,,Hamburg,2025-02-16 20:49:06
2,2350-2800-eur-telefonischer-kundenbetreuer-kun...,W&W Human Resources - Ivonne Wresch -,2350-2800 € Telefonischer Kundenbetreuer m/w/...,<p>Möchten Sie als Telefonischer Kundenbetreue...,False,https://www.arbeitnow.com/jobs/companies/ww-hu...,Customer Service,,Berlin,2025-02-16 20:49:06
3,ugc-creator-fur-tiktok-i-zeig-der-welt-die-lec...,Global Treats GmbH,UGC Creator für TikTok I Zeig der Welt die lec...,<p>Vernasche die Welt ist die angesagte Marke ...,True,https://www.arbeitnow.com/jobs/companies/globa...,"Remote, Marketing and Communication",,Wutöschingen,2025-02-16 20:49:06
4,customer-service-representative-dusseldorf-473465,Japan Management,Customer Service Representative (m/f/d),<h2>Tasks</h2>\n<p>✅ Communicate with customer...,False,https://www.arbeitnow.com/jobs/companies/japan...,"CRM, Direct Marketing",entry,Düsseldorf,2025-02-16 19:49:05
...,...,...,...,...,...,...,...,...,...,...
95,gebietsleiter-in-diez-116660,letswerk - Matthias Fränz,Gebietsleiter/-in (m/w/d) Diez,<p>Wir sind eine bundesweit erfolgreich agiere...,False,https://www.arbeitnow.com/jobs/companies/letsw...,Branch Management,,Diez,2025-02-15 00:49:04
96,gebietsleiter-in-giessen-giessen-237943,letswerk - Matthias Fränz,Gebietsleiter/-in (m/w/d) Gießen,<p>Wir sind eine bundesweit erfolgreich agiere...,False,https://www.arbeitnow.com/jobs/companies/letsw...,Branch Management,,Giessen,2025-02-15 00:49:04
97,gebietsleiter-in-montabaur-180212,letswerk - Matthias Fränz,Gebietsleiter/-in (m/w/d) Montabaur,<p>Wir sind eine bundesweit erfolgreich agiere...,False,https://www.arbeitnow.com/jobs/companies/letsw...,Branch Management,,Montabaur,2025-02-15 00:49:04
98,junior-it-systemadministrator-wiesbaden-136146,PIMA Health Group GmbH,Junior IT-Systemadministrator (w/m/d),<p>In dieser Rolle als Junior IT-Systemadminis...,False,https://www.arbeitnow.com/jobs/companies/pima-...,System and Network Administration,berufseinstieg,Wiesbaden,2025-02-15 00:49:04


In [34]:
# Database connection parameters
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "6851"
DB_HOST = "localhost"  # 127.0.0.1 for local database
DB_PORT = "5432"  # Default PostgreSQL port

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    
    print("Connected to PostgreSQL successfully!")

    # Create a cursor object to execute SQL queries
    cur = conn.cursor()

    create_table_query = """
CREATE TABLE IF NOT EXISTS jobs (
    id SERIAL PRIMARY KEY,
    slug VARCHAR(255) UNIQUE,
    company_name VARCHAR(255),
    job_title VARCHAR(255),
    description TEXT,
    remote BOOLEAN,
    job_url TEXT,
    tags TEXT,  -- Store multiple tags as JSON or text
    job_types VARCHAR(100),  -- Nullable, as some values are NaN
    location VARCHAR(255),
    created_at TIMESTAMP
);

"""
    cur.execute(create_table_query)
    conn.commit()
    print("Table 'users' created successfully!")


    # Execute a query
    cur.execute("SELECT version();")
    
    # Fetch and print the result
    db_version = cur.fetchone()
    print("PostgreSQL Version:", db_version)

    # Close the connection
    cur.close()
    conn.close()

except Exception as e:
    print("Error connecting to PostgreSQL:", e)

Connected to PostgreSQL successfully!
Table 'users' created successfully!
PostgreSQL Version: ('PostgreSQL 17.3 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit',)
